In [3]:
url = "https://docs.smith.langchain.com/observability"

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

## langsmith Tracking 
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(url)

In [7]:
loader

In [8]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/observability', 'title': 'Observability Quick Start | 🦜️🛠️ LangSmith', 'description': 'This tutorial will get you up and running with our observability SDK by showing you how to', 'language': 'en'}, page_content='\n\n\n\n\nObservability Quick Start | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith. Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityQuick StartTutorialsAdd observability to your LLM applicationHow-to GuidesAnnotate code for tracingFilter traces in the applicationUpload files with tracesDashboardsLog traces to specific projectSet up automation rulesOnline EvaluationSet a sampling rate for tracesAdd metadata and tags to tracesImplement distributed tracingAccess the current run (span) within a traced functionLog multimodal tracesLog retriever tracesLog custom LLM tracesPrevent logging 

In [9]:
## Text Chunk 
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [12]:
final_docs = text_splitter.split_documents(docs)

In [13]:
len(final_docs)

17

In [14]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [15]:
## local vector stroe DB
from langchain_community.vectorstores import FAISS

vectorStore = FAISS.from_documents(final_docs, embeddings)

In [16]:
vectorStore

In [22]:
## Query From a vector db
query = "We will instrument a simple RAG application for this tutorial, but feel free to use your own code if you'd like - just make sure it has an LLM call!"

result = vectorStore.similarity_search(query)
result[0].page_content

"4. Define your application\u200b\nWe will instrument a simple RAG\napplication for this tutorial, but feel free to use your own code if you'd like - just make sure\nit has an LLM call!"

In [25]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000027D63602CB0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000027D653A7D30>, root_client=<openai.OpenAI object at 0x0000027D6516D2D0>, root_async_client=<openai.AsyncOpenAI object at 0x0000027D636013C0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [24]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
        Answer the following questiobn based only on the provided context

        <context>
        {context} 
        </context>
    """
)
prompt

ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        Answer the following questiobn based only on the provided context\n\n        <context>\n        {context} \n        </context>\n    '), additional_kwargs={})])

In [26]:
document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        Answer the following questiobn based only on the provided context\n\n        <context>\n        {context} \n        </context>\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000027D63602CB0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000027D653A7D30>, root_client=<openai.OpenAI object at 0x0000027D6516D2D0>, root_async_client=<openai.AsyncOpenAI object at 0x0000027D636013C0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| 

In [27]:
from langchain_core.documents import Document

response = document_chain.invoke({
    "input":"Trace OpenAI calls",
    "context":[Document(page_content="The first thing you might want to trace is all your OpenAI calls. LangSmith makes this easy with the wrap_openai (Python) or wrapOpenAI (TypeScript) wrappers. All you have to do is modify your code to use the wrapped client instead of using the OpenAI client directly.")]
})

In [28]:
response

'To trace your OpenAI calls, you can use the LangSmith wrappers: `wrap_openai` for Python or `wrapOpenAI` for TypeScript. Modify your code to use the wrapped client instead of the OpenAI client directly.'

In [29]:
## Retriever -> interface no need to do similarity serach

vectorStore

In [30]:
retriever = vectorStore.as_retriever()

In [31]:
from langchain.chains import create_retrieval_chain

retrieve_chain = create_retrieval_chain(retriever, document_chain)

In [33]:
retrieve_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000027D7ED5D720>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        Answer the following questiobn based only on the provided context\n\n        <context>\n        {context} \n        </context>\n    '), add

In [35]:
response = retrieve_chain.invoke({
    "input":"Trace OpenAI calls"
})

response['answer']

"How can you trace OpenAI calls using LangSmith according to the provided context?\n\nYou can trace OpenAI calls using LangSmith by modifying your code to use the `wrap_openai` (Python) or `wrapOpenAI` (TypeScript) wrappers. By doing this, you'll replace the direct use of the OpenAI client with the wrapped client, enabling the tracing of OpenAI calls."

In [36]:
response

{'input': 'Trace OpenAI calls',
 'context': [Document(id='96a99d78-0308-4480-9dd9-3944ac4a7d05', metadata={'source': 'https://docs.smith.langchain.com/observability', 'title': 'Observability Quick Start | 🦜️🛠️ LangSmith', 'description': 'This tutorial will get you up and running with our observability SDK by showing you how to', 'language': 'en'}, page_content='5. Trace OpenAI calls\x00\x00\u200b\nThe first thing you might want to trace is all your OpenAI calls. LangSmith makes this easy with the wrap_openai (Python) or wrapOpenAI (TypeScript) wrappers.\nAll you have to do is modify your code to use the wrapped client instead of using the OpenAI client directly.'),
  Document(id='30bea452-8618-42a9-b43a-35ad22f52ac8', metadata={'source': 'https://docs.smith.langchain.com/observability', 'title': 'Observability Quick Start | 🦜️🛠️ LangSmith', 'description': 'This tutorial will get you up and running with our observability SDK by showing you how to', 'language': 'en'}, page_content='Now i